In [1]:
info_root = 'dataset/RealEstate10K'
mode = 'test'
data_root = 'dataset/RealEstate_raw_data/test/test'


In [2]:
from dataset.dataset import *

ds = RealEstate_dataset(info_root, data_root, mode)

c:\Users\Administrator\anaconda3\envs\zero123\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from torch.utils.data import DataLoader

batch_size = 1
loader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=0)

for idx, data in enumerate(loader):
    print(data[0].shape, data[1].shape, data[2].shape, data[3].shape, data[4].shape)
    break


dataset/RealEstate10K/test//0017ce4c6a39d122.txt
torch.Size([1, 3, 128, 128]) torch.Size([1, 3, 128, 128]) torch.Size([1, 3, 3]) torch.Size([1, 3, 3]) torch.Size([1, 3])
